In [1]:
import $ivy.`org.apache.spark::spark-sql:3.2.0` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.10.1`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = SparkSession
      .builder()
      .config("spark.sql.shuffle.partitions", 4)
      .master("local[4]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/05/12 09:31:10 INFO SparkContext: Running Spark version 3.2.0
22/05/12 09:31:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/12 09:31:10 INFO ResourceUtils: ==============================================================
22/05/12 09:31:10 INFO ResourceUtils: No custom resources configured for spark.driver.
22/05/12 09:31:10 INFO ResourceUtils: ==============================================================
22/05/12 09:31:10 INFO SparkContext: Submitted application: 8faf069a-b59e-435e-882d-ea90e5681f34
22/05/12 09:31:10 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
22/05/1

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                               


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@7dfd704f
import spark.implicits._



In [2]:
val races = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/races.csv")
    .select("raceId", "year")

races: DataFrame = [raceId: string, year: string]

In [3]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window

In [4]:
val seasonWindow = Window.partitionBy("year")
val driverRaceWindow = Window.partitionBy("driverId", "raceId")
val raceDriverLapWindow = Window.partitionBy("driverId", "raceId").orderBy("lap")

seasonWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@248716
driverRaceWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@7c18ec24
raceDriverLapWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@69d80181

In [5]:
val overtakes = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/lap_times.csv")
    .withColumn("position", col("position").cast(IntegerType)) 
    .withColumn("lap", col("lap").cast(IntegerType)) 
    .join(races, "raceId")
    .withColumn("positionNextLap", lead(col("position"), 1).over(raceDriverLapWindow))
    .withColumn("positionsGainedLap", when(col("positionNextLap") < col("position") , abs(col("position") - col("positionNextLap"))).otherwise(0))
    .groupBy("raceId", "driverId")
    .agg(sum(col("positionsGainedLap")).alias("positionsGained"), first(col("year")).alias("year"))
    .groupBy("year")
    .agg(sum(col("positionsGained")).alias("positionsGainedSeason"))
    .withColumn("rankPositionsGained", rank().over(Window.orderBy(col("positionsGainedSeason").desc)))

overtakes: DataFrame = [year: string, positionsGainedSeason: bigint ... 1 more field]

In [6]:
val leadersTroughoutSeason = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/driver_standings.csv")
    .join(races, "raceId")
    .where(col("position") === 1)
    .dropDuplicates("driverId", "position", "year")
    .groupBy("year")
    .agg(approx_count_distinct(col("driverId")).alias("distinctLeaders"))
    .withColumn("rankDistinctLeaders", rank().over(Window.orderBy(col("distinctLeaders").desc)))

leadersTroughoutSeason: DataFrame = [year: string, distinctLeaders: bigint ... 1 more field]

In [7]:
val winnersTroughoutSeason = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/results.csv")
    .join(races, "raceId")
    .where(col("position") === 1)
    .dropDuplicates("driverId", "position", "year")
    .groupBy("year")
    .agg(approx_count_distinct(col("driverId")).alias("distinctWinners"))
    .sort(col("distinctWinners").desc)
    .withColumn("rankDistinctWinners", rank().over(Window.orderBy(col("distinctWinners").desc)))

winnersTroughoutSeason: DataFrame = [year: string, distinctWinners: bigint ... 1 more field]

In [8]:
val averageRank = udf((cols: Seq[Int]) => {cols.sum / cols.size}: Double)
spark.udf.register("averageRank", averageRank)

averageRank: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cmd7$Helper$$Lambda$5373/1093631140@6e017788,
  DoubleType,
  List(
    Some(
      ExpressionEncoder(
        NewInstance(
          class org.apache.spark.sql.catalyst.util.GenericArrayData,
          List(
            BoundReference(0, ObjectType(interface scala.collection.Seq), true)
          ),
          List(),
          true,
          ArrayType(IntegerType, false),
          None
        ),
        UnresolvedMapObjects(
          org.apache.spark.sql.catalyst.ScalaReflection$$$Lambda$5380/1908438313@f4a956d,
          GetColumnByOrdinal(0, ArrayType(IntegerType, false)),
          Some(interface scala.collection.Seq)
        ),
        scala.collection.Seq
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, DoubleType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, DoubleType),
          DoubleType,
          List("- root class: \"scala.

In [9]:
val results = overtakes
    .join(leadersTroughoutSeason, Seq("year"), "inner")
    .join(winnersTroughoutSeason, Seq("year"), "inner")
    .withColumn("avgRank", averageRank(array(col("rankDistinctWinners"), col("rankDistinctLeaders"), col("rankPositionsGained"))))
    .withColumn("overallRank", rank().over(Window.orderBy("avgRank")))

results: DataFrame = [year: string, positionsGainedSeason: bigint ... 7 more fields]

In [12]:
results.select("year", "overallRank", "positionsGainedSeason", "distinctLeaders", "distinctWinners").show(70, false)

+----+-----------+---------------------+---------------+---------------+
|year|overallRank|positionsGainedSeason|distinctLeaders|distinctWinners|
+----+-----------+---------------------+---------------+---------------+
|2012|1          |5077                 |4              |8              |
|2008|2          |3123                 |4              |7              |
|2003|3          |2955                 |3              |8              |
|1997|4          |2642                 |3              |6              |
|2010|5          |2747                 |6              |5              |
|1999|6          |2151                 |3              |6              |
|2021|7          |2901                 |2              |6              |
|2013|8          |4697                 |2              |5              |
|2019|9          |3201                 |2              |5              |
|2007|10         |2974                 |3              |4              |
|2018|11         |2692                 |2          

In [10]:
def run[A](code: => A): Long = {
    val start = System.currentTimeMillis()
    val res = code
    System.currentTimeMillis() - start
}

def average(list: List[Long]):Long = list.sum / list.size.toLong

def timeTest(f: => Long): Long = {
    val list = (1 to 1).map(_ => f).toList
    average(list)
}

val res = timeTest(run(results.collect))

defined function run
defined function average
defined function timeTest
res: Long = 2390L